In [48]:
import paho.mqtt.client as mqtt
import json
import requests

In [46]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("$SYS/#")
    client.subscribe("frigate/events")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    if (msg.topic == 'frigate/events'):
        payload = json.loads(str(msg.payload)[2:-1])
        print(f'{payload["after"]["id"]} {payload["after"]["end_time"]}')
    else:
        print(msg.topic+" "+str(msg.payload))

mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.connect("localhost", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
mqttc.loop_forever()

Connected with result code Success
1710880842.594859-kakz56 None
1710880842.594859-1adm48 None
1710880842.594859-kakz56 None
1710880842.594859-kakz56 None
1710880842.594859-1adm48 None
1710880842.594859-1adm48 None
1710880842.594859-kakz56 1710880857.996247
1710880842.594859-1adm48 1710880857.996247
$SYS/brokers/disconnected b'{"username":"(null)","ts":1710880888676,"reason_code":"6","client_id":"nanomq-80ca3a4d"}'


KeyboardInterrupt: 

In [63]:
"""
1710880842.594859-kakz56
"""

'\n1710880842.594859-kakz56\n'

In [44]:
id = '1710880842.594859-kakz56'

In [42]:
response = requests.get(f'http://localhost:5000/api/events/{id}/clip.mp4')

with open(f'{id}.mp4', 'wb') as video:
  video.write(response.content)

In [43]:
# Тута определение скорости

max_speed = 40
if max_speed > 30:
    url = f'http://localhost:5000/api/events/{id}/sub_label'
    data = {
        "subLabel": f"Макс. скорость: {max_speed} км/ч"
    }

    response = requests.post(url=url, json=data)
    response.json()

{'message': 'Event 1710880842.594859-kakz56 sub label set to Макс. скорость: 40 км/ч',
 'success': True}

### Заготовка

In [49]:
import paho.mqtt.client as mqtt
import json
import requests

In [47]:
def speed_estimation(id):
  response = requests.get(f'http://localhost:5000/api/events/{id}/clip.mp4')

  with open(f'{id}.mp4', 'wb') as video:
    video.write(response.content)

  # Тута определение скорости
  max_speed = 40

  if max_speed > 30:
      url = f'http://localhost:5000/api/events/{id}/sub_label'
      data = {
          "subLabel": f"Макс. скорость: {max_speed} км/ч"
      }

      response = requests.post(url=url, json=data)
      response.json()

In [ ]:
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, reason_code, properties):
    print(f"Connected with result code {reason_code}")
    # Subscribing in on_connect()
    client.subscribe("frigate/events")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    if (msg.topic == 'frigate/events'):
        payload = json.loads(str(msg.payload)[2:-1])
        id = payload["after"]["id"]
        end_time = payload["after"]["end_time"]
        print(f'{msg.topic} {id} {end_time}')

        if end_time is not None:
            speed_estimation(id)
    else:
        print(msg.topic+" "+str(msg.payload))

mqttc = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
mqttc.on_connect = on_connect
mqttc.on_message = on_message

mqttc.connect("localhost", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
mqttc.loop_forever()